<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_07_2_train_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [25]:
!pip uninstall jax jaxlib -y


Found existing installation: jax 0.3.10
Uninstalling jax-0.3.10:
  Successfully uninstalled jax-0.3.10
Found existing installation: jaxlib 0.3.10+cuda11.cudnn805
Uninstalling jaxlib-0.3.10+cuda11.cudnn805:
  Successfully uninstalled jaxlib-0.3.10+cuda11.cudnn805


In [26]:
!pip install "jax[cuda11_cudnn805]==0.3.10" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
  Using cached jax-0.3.10-py3-none-any.whl
  Using cached https://storage.googleapis.com/jax-releases/cuda11/jaxlib-0.3.10%2Bcuda11.cudnn805-cp310-none-manylinux2014_x86_64.whl (175.7 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.7 requires jax>=0.4.6, but you have jax 0.3.10 which is incompatible.
flax 0.6.9 requires jax>=0.4.2, but you have jax 0.3.10 which is incompatible.
orbax-checkpoint 0.2.1 requires jax>=0.4.8, but you have jax 0.3.10 which is incompatible.
tensorflow 2.12.0 requires jax>=0.3.15, but you have jax 0.3.10 which is incompatible.


In [27]:
!pip install torch==1.8.1 torchvision==0.9.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement torch==1.8.1 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.8.1


In [28]:
!pip install torch torchvision


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-2.0.1-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)


In [30]:
!pip install torch==1.9.0 torchvision==0.10.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement torch==1.9.0 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.9.0


In [31]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git


fatal: destination path 'stylegan2-ada-pytorch' already exists and is not an empty directory.


In [32]:
!pip install ninja

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
!ls /content/drive/MyDrive/data/gan/images/Abstract_gallery_2

Abstract_image_2782.jpg  Abstract_image_2812.jpg  Abstract_image_2842.jpg
Abstract_image_2783.jpg  Abstract_image_2813.jpg  Abstract_image_2843.jpg
Abstract_image_2784.jpg  Abstract_image_2814.jpg  Abstract_image_2844.jpg
Abstract_image_2785.jpg  Abstract_image_2815.jpg  Abstract_image_2845.jpg
Abstract_image_2786.jpg  Abstract_image_2816.jpg  Abstract_image_2846.jpg
Abstract_image_2787.jpg  Abstract_image_2817.jpg  Abstract_image_2847.jpg
Abstract_image_2788.jpg  Abstract_image_2818.jpg  Abstract_image_2848.jpg
Abstract_image_2789.jpg  Abstract_image_2819.jpg  Abstract_image_2849.jpg
Abstract_image_2790.jpg  Abstract_image_2820.jpg  Abstract_image_2850.jpg
Abstract_image_2791.jpg  Abstract_image_2821.jpg  Abstract_image_2851.jpg
Abstract_image_2792.jpg  Abstract_image_2822.jpg  Abstract_image_2852.jpg
Abstract_image_2793.jpg  Abstract_image_2823.jpg  Abstract_image_2853.jpg
Abstract_image_2794.jpg  Abstract_image_2824.jpg  Abstract_image_2854.jpg
Abstract_image_2795.jpg  Abstract_imag

In [34]:
CMD = "python /content/stylegan2-ada-pytorch/dataset_tool.py "\
  "--source /content/drive/MyDrive/data/gan/images/Abstract_gallery_2 "\
  "--dest /content/drive/MyDrive/data/gan/dataset/circuit5"

!{CMD}

100% 90/90 [00:11<00:00,  7.66it/s]


In [35]:
#!rm -R /content/drive/MyDrive/data/gan/dataset/circuit/*

In [36]:
from os import listdir
from os.path import isfile, join
import os
from PIL import Image
from tqdm.notebook import tqdm

IMAGE_PATH = '/content/drive/MyDrive/data/gan/images/Abstract_gallery_2'
files = [f for f in listdir(IMAGE_PATH) if isfile(join(IMAGE_PATH, f))]

base_size = None
for file in tqdm(files):
  file2 = os.path.join(IMAGE_PATH,file)
  img = Image.open(file2)
  sz = img.size
  if base_size and sz!=base_size:
    print(f"Inconsistant size: {file2}")
  elif img.mode!='RGB':
    print(f"Inconsistant color format: {file2}")
  else:
    base_size = sz


  0%|          | 0/90 [00:00<?, ?it/s]

## Perform Initial Training

This code performs the initial training.  Set SNAP low enough to get a snapshot before Colab forces you to quit.

In [37]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments/x"
DATA = "/content/drive/MyDrive/data/gan/dataset/circuit5"
SNAP = 10

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py "\
  f"--snap {SNAP} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 10,
  "network_snapshot_ticks": 10,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/drive/MyDrive/data/gan/dataset/circuit5",
    "use_labels": false,
    "max_size": 90,
    "xflip": false,
    "resolution": 1024
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768

## Resume Training

You can now resume training after you are interrupted by something in the pervious step.

In [38]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
NETWORK = "network-snapshot-000100.pkl"
RESUME = os.path.join(EXPERIMENTS, \
                "00008-circuit-auto1-resumecustom", NETWORK)
DATA = "/content/drive/MyDrive/data/gan/dataset/circuit4"
SNAP = 10

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py "\
  f"--snap {SNAP} --resume {RESUME} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 10,
  "network_snapshot_ticks": 10,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/drive/MyDrive/data/gan/dataset/circuit4",
    "use_labels": false,
    "max_size": 90,
    "xflip": false,
    "resolution": 1024
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768